In [1]:
import requests
env = open('../.env').readlines()
IS_PROD = True
env_prefix = "" if IS_PROD else "DEV_"
DISCORD_APPLICATION_ID = [x.split('=')[1].strip().replace('"', '') for x in env if x.startswith(f'{env_prefix}DISCORD_APPLICATION_ID')][0]
DISCORD_BOT_TOKEN = [x.split('=')[1].strip().replace('"', '') for x in env if x.startswith(f'{env_prefix}DISCORD_BOT_TOKEN')][0]
headers = {
    "Authorization": f"Bot {DISCORD_BOT_TOKEN}"
}

# LIST COMMANDS

In [2]:
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"
r = requests.get(url, headers=headers)
r.json()

[{'id': '912067168210931783',
  'application_id': '912000855186751489',
  'version': '926834491530575894',
  'default_permission': True,
  'default_member_permissions': None,
  'type': 1,
  'name': 'starter',
  'description': 'Generate a conversation starter given some topics',
  'dm_permission': None,
  'options': [{'type': 3,
    'name': 'topics',
    'description': "List of topics separated by ','. Example 'ice breaker,travel,dating'"},
   {'type': 3,
    'name': 'players',
    'description': "List of challenged players separated by ','. Example '@user1,@user2'"}]}]

# /starter

In [3]:
d = {
    "name": "starter",
    "description": "Generate a conversation starter given some topics",
    "options": [
        {
            "name": "topics",
            "description": "List of topics separated by ','."
            + " Example 'ice breaker,travel,dating'. (default: 'ice breaker')",
            "type": 3,
            "required": False,
        },
        {
            "name": "players",
            "description": "List of challenged players separated by ','. Example '@user1,@user2'",
            "type": 3,
            "required": False,
        }
    ]
}
COMMAND_ID="912067168210931783"
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands/"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '912067168210931783',
 'application_id': '912000855186751489',
 'version': '937655825864998973',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'starter',
 'name_localizations': None,
 'description': 'Generate a conversation starter given some topics',
 'description_localizations': None,
 'dm_permission': None,
 'options': [{'type': 3,
   'name': 'topics',
   'name_localizations': None,
   'description': "List of topics separated by ','. Example 'ice breaker,travel,dating'. (default: 'ice breaker')",
   'description_localizations': None},
  {'type': 3,
   'name': 'players',
   'name_localizations': None,
   'description': "List of challenged players separated by ','. Example '@user1,@user2'",
   'description_localizations': None}]}

# COMMAND DELETE

In [13]:
COMMAND_ID = ""
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands/{COMMAND_ID}"
r = requests.delete(url, headers=headers)
r.text

''

# /about

Langame
A game where you can play with your friends.
How to play
Send this command to a channel:
/starter topics:ice breaker,travel,whatever topic your like

You can also add players to the Langame by adding the following option:
/starter players:@user1,@user2,@user3

With love, by https://langa.me/. Augmented conversations.


In [5]:

d = {
    "name": "about",
    "description": "Give information about the bot",
}
COMMAND_ID = ""
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '937655936204562453',
 'application_id': '912000855186751489',
 'version': '937655936204562454',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'about',
 'name_localizations': None,
 'description': 'Give information about the bot',
 'description_localizations': None,
 'dm_permission': None}

# /sub

/sub topics:ice breaker,travel players:@louis030195,@bob27 frequency:12h

results in the bot posting conversation starters between these two every 12h on the given topics

Optionally use players:random to have the bot choose a random player from the server


### Execution

Topics:ice breaker,travel
Players:@louis030195,@bob27

What is your favorite place?

CTA component button to stop the sub, like button click on /unsub whatever

In [6]:
d = {
    "name": "sub",
    "description": "Subscribe to frequent Langames",
    "options": [
        {
            "name": "topics",
            "description": "List of topics separated by ','. Example 'ice breaker,travel,dating'"
            + "(default 'ice breaker')",
            "type": 3,
            "required": False,
        },
        # {
        #     "name": "players",
        #     "description": "List of challenged players separated by ','."
        #     + " Example '@user1,@user2'. (default is random players)",
        #     "type": 3,
        #     "required": False,
        # },
        {
            "name": "frequency",
            "description": "Frequency in minutes,"
            + " hours or days. Example '30m' or '5h' or '2d'"
            + " (default is 3h)",
            "type": 3,
            "required": False,
        }
    ],
}
COMMAND_ID = ""
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '937656007562240060',
 'application_id': '912000855186751489',
 'version': '937656007562240061',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'sub',
 'name_localizations': None,
 'description': 'Subscribe to frequent Langames',
 'description_localizations': None,
 'dm_permission': None,
 'options': [{'type': 3,
   'name': 'topics',
   'name_localizations': None,
   'description': "List of topics separated by ','. Example 'ice breaker,travel,dating'(default 'ice breaker')",
   'description_localizations': None},
  {'type': 3,
   'name': 'frequency',
   'name_localizations': None,
   'description': "Frequency in minutes, hours or days. Example '30m' or '5h' or '2d' (default is 3h)",
   'description_localizations': None}]}

# /unsub

Unsubscribe the server from Langames.

In [7]:
d = {
    "name": "unsub",
    "description": "Unsubscribe from frequent Langames",
}
COMMAND_ID = None
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '937656041238315028',
 'application_id': '912000855186751489',
 'version': '937656041238315029',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'unsub',
 'name_localizations': None,
 'description': 'Unsubscribe from frequent Langames',
 'description_localizations': None,
 'dm_permission': None}

# Other

In [11]:
# url = f"https://discord.com/api/v8/guilds/888438975646285834/members/267955006974590976"
# url = f"https://discord.com/api/v8/guilds/888438975646285834/roles"
url = f"https://discord.com/api/v8/guilds/888438975646285834/members?limit=1000"
r = requests.get(url, headers=headers)
r.json()

[{'roles': ['933033976828948520', '933032577319731292'],
  'nick': None,
  'avatar': None,
  'premium_since': None,
  'joined_at': '2022-01-18T16:18:21.437000+00:00',
  'is_pending': False,
  'pending': False,
  'communication_disabled_until': None,
  'user': {'id': '172002275412279296',
   'username': 'Tatsu',
   'avatar': '43fc1879641f6239d6e4038eaa7dabbc',
   'discriminator': '8792',
   'public_flags': 589824,
   'bot': True},
  'mute': False,
  'deaf': False},
 {'roles': ['916740688396292216', '935092248167739432'],
  'nick': None,
  'avatar': None,
  'premium_since': None,
  'joined_at': '2021-09-17T14:59:18.242000+00:00',
  'is_pending': False,
  'pending': False,
  'communication_disabled_until': None,
  'user': {'id': '267955006974590976',
   'username': 'louis030195',
   'avatar': '1a8d658e1267a1b0b36f93c151e4c733',
   'discriminator': '2462',
   'public_flags': 0},
  'mute': False,
  'deaf': False},
 {'roles': ['933033976828948520', '935092248167739432'],
  'nick': None,
  'a